In [1]:
import json
import pandas as pd
import numpy as np 
from datetime import datetime,timezone
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import zstandard
import os
import sys
import logging.handlers
import re
from tqdm import tqdm
import csv

In [2]:
post_df = pd.read_csv('posts_raw_15_09_24.csv')

In [3]:
post_df.head()

,selftext,distinguished,created_utc,id,upvote_ratio,num_comments,title,score
0,[deleted],NaN,1606780814,k48sqm,1.00,0,Onto something new,1
1,[deleted],NaN,1606780839,k48t1v,1.00,8,How to get into stocks as a poor/broke person.,1
2,[removed],NaN,1606780873,k48tgg,0.67,3,Cannabis vote,1
3,Every time a new submission is posted to walls...,NaN,1606780909,k48tvz,0.81,126,WSBVoteBot Log for Dec 01 2020,6
4,[deleted],NaN,1606780924,k48u34,1.00,0,I lost hope,1


In [4]:
post_df.dtypes

selftext          object
distinguished     object
created_utc        int64
id                object
upvote_ratio     float64
num_comments       int64
title             object
score              int64
dtype: object

In [5]:
post_df.shape

(894803, 8)

In [6]:
post_df = post_df.sort_values(by='created_utc', ascending=True).reset_index(drop=True)
post_df.head()

,selftext,distinguished,created_utc,id,upvote_ratio,num_comments,title,score
0,[deleted],NaN,1606780814,k48sqm,1.00,0,Onto something new,1
1,[deleted],NaN,1606780839,k48t1v,1.00,8,How to get into stocks as a poor/broke person.,1
2,[removed],NaN,1606780873,k48tgg,0.67,3,Cannabis vote,1
3,Every time a new submission is posted to walls...,NaN,1606780909,k48tvz,0.81,126,WSBVoteBot Log for Dec 01 2020,6
4,[deleted],NaN,1606780924,k48u34,1.00,0,I lost hope,1


In [7]:
post_df = post_df[(post_df['distinguished'] != 'moderator') & (post_df['distinguished'] != 'admin')]
post_df.drop('distinguished', axis=1, inplace = True)

In [8]:
from zoneinfo import ZoneInfo
post_df['created_et_date'] = post_df['created_utc'].apply(lambda x: datetime.fromtimestamp(x, tz=timezone.utc).
                                                          astimezone(ZoneInfo('America/New_York')).strftime("%Y-%m-%d"))
post_df['created_et_date'] = pd.to_datetime(post_df['created_et_date'])

post_df['created_et_timestamp'] = post_df['created_utc'].apply(lambda x: datetime.fromtimestamp(x, tz=timezone.utc).
                                                               astimezone(ZoneInfo('America/New_York')).strftime("%Y-%m-%d %H:%M:%S %Z"))

post_df['created_et_time'] = post_df['created_utc'].apply(lambda x: datetime.fromtimestamp(x, tz=timezone.utc)
                        .astimezone(ZoneInfo('America/New_York')).strftime("%H:%M:%S"))

post_df['created_et_timestamp'] = post_df['created_et_timestamp'].str.replace(r'\s\w+$', '', regex=True)  
post_df['created_et_timestamp'] = pd.to_datetime(post_df['created_et_timestamp']).dt.tz_localize(ZoneInfo("America/New_York"))

#post_df.drop('created_utc', axis=1, inplace=True)
post_df = post_df.sort_values(by ='created_et_timestamp', ascending = True).reset_index(drop = True)

In [9]:
post_df.dtypes

selftext                                          object
created_utc                                        int64
id                                                object
upvote_ratio                                     float64
num_comments                                       int64
title                                             object
score                                              int64
created_et_date                           datetime64[ns]
created_et_timestamp    datetime64[ns, America/New_York]
created_et_time                                   object
dtype: object

In [10]:
post_df.shape

(894726, 10)

In [11]:
#removing removed and deleted posts - 
post_df['title'] = post_df['title'].replace({'[removed]': '', '[deleted]': '','[deleted by user]': ''})
post_df['selftext'] = post_df['selftext'].replace({'[removed]': '', '[deleted]': '','[deleted by user]': ''})

In [12]:
#appending self text to title for complete post 
post_df['fulltext'] = post_df['title'] + ' ' + post_df['selftext']
post_df['fulltext'] = post_df['title'].str.strip()
post_df = post_df.drop(['selftext', 'title'], axis=1)

In [13]:
post_df.head()

,created_utc,id,upvote_ratio,num_comments,score,created_et_date,created_et_timestamp,created_et_time,fulltext
0,1606780814,k48sqm,1.00,0,1,2020-11-30,2020-11-30 19:00:14-05:00,19:00:14,Onto something new
1,1606780839,k48t1v,1.00,8,1,2020-11-30,2020-11-30 19:00:39-05:00,19:00:39,How to get into stocks as a poor/broke person.
2,1606780873,k48tgg,0.67,3,1,2020-11-30,2020-11-30 19:01:13-05:00,19:01:13,Cannabis vote
3,1606780909,k48tvz,0.81,126,6,2020-11-30,2020-11-30 19:01:49-05:00,19:01:49,WSBVoteBot Log for Dec 01 2020
4,1606780924,k48u34,1.00,0,1,2020-11-30,2020-11-30 19:02:04-05:00,19:02:04,I lost hope


In [14]:
#removing bot comments - 
import pandas as pd

def drop_rows_keywords(post_df, keywords):
    pattern = '|'.join(keywords)
    post_df = post_df[~post_df['fulltext'].str.contains(pattern, case=False, na=False)]
    return post_df

keywords_to_remove = ['WSBVoteBot', 'Cannabis', '\btest\b', 'pickbot','Bot Action','Python trading bot',
                'mining bot','trading bot','BOTS ARE EVERYWHERE, CHECK THE ACCOUNT THAT’S POSTING FFS',
                'ban this bot','A Telegram bot to poll']
post_df = drop_rows_keywords(post_df, keywords_to_remove)


post_df.shape


(893188, 9)

In [15]:
post_df['fulltext'] = post_df['fulltext'].astype(str)

In [16]:
#removing links from text
def remove_urls(text):
 
    url_pattern = r'(https?:\/\/(?:[-\w]+\.)+[a-zA-Z]{2,}(?:\/[^\s]*)?)'
    return re.sub(url_pattern, '', text)

post_df['fulltext'] = post_df['fulltext'].apply(remove_urls)

In [17]:
#dropping empty and few char rows - give better prompt
import emoji
def non_emoji(text):
    if len(text) <= 2 and not emoji.is_emoji(text):
        return True
    return False

post_df = post_df[~(post_df['fulltext'].apply(non_emoji))]
post_df.shape


(890390, 9)

In [18]:
# dropping duplicates where char length is greater than average length of the post 
df_comments_zero = post_df[(post_df['num_comments'] == 0) & (post_df['fulltext'].str.len() >= 100)]
df_comments_nonzero = pd.merge(post_df, df_comments_zero, how='outer', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)
df_comments_zero = df_comments_zero.drop_duplicates(subset=['fulltext'])
post_df = pd.concat([df_comments_zero, df_comments_nonzero]).reset_index(drop=True)
post_df.shape

(887628, 9)

In [19]:
post_df.to_csv('out_data/posts output folder/first_clean_posts_18_09_2024.csv', index=False)

In [20]:
#creating columns for trade time and trade date 
non_trade_dates = ['2020-12-25', '2021-01-01', '2021-01-18', '2021-02-15', '2020-12-05','2020-12-06','2020-12-12','2020-12-13',
                   '2020-12-19','2020-12-20','2020-12-26','2020-12-27','2021-01-02','2021-01-03','2021-01-09','2021-01-10',
                   '2021-01-16','2021-01-17', '2021-01-23','2021-01-24','2021-01-30','2021-01-31', '2021-02-06','2021-02-07',
                   '2021-02-13','2021-02-14']

non_trade_dates = pd.to_datetime(non_trade_dates)

post_df['is_trade_day'] = post_df['created_et_date'].apply(lambda x: 0 if x in non_trade_dates else 1)
            
#marking comments created outside the trading hours - 
start_time = pd.to_datetime('09:30:00').time()  # 9:30 AM ET 
end_time = pd.to_datetime('16:00:00').time()  # 4:00 PM ET 
post_df['created_et_time'] = pd.to_datetime(post_df['created_et_time'], format='%H:%M:%S').dt.time
post_df['is_trade_time'] = post_df['created_et_time'].apply(lambda x: 1 if start_time <= x <= end_time else 0)


In [21]:
post_df.sample(5)

,created_utc,id,upvote_ratio,num_comments,score,created_et_date,created_et_timestamp,created_et_time,fulltext,is_trade_day,is_trade_time
661521,1612228298,lajomh,1.0,0,1,2021-02-01,2021-02-01 20:11:38-05:00,20:11:38,Good to Know Guys !! We hold strong 💪💪💪,1,0
139654,1611487066,l3xnj6,1.0,0,1,2021-01-24,2021-01-24 06:17:46-05:00,06:17:46,LEMON HUNTING SEASON!!! 🏹🍋,0,0
18014,1611851453,l71g0o,1.0,0,1,2021-01-28,2021-01-28 11:30:53-05:00,11:30:53,If the hedge funds gets away with this I can p...,1,1
862128,1613055478,lhmdm4,1.0,0,1,2021-02-11,2021-02-11 09:57:58-05:00,09:57:58,Please stop only upvoting memes,1,1
786010,1612480756,lct7n5,1.0,0,1,2021-02-04,2021-02-04 18:19:16-05:00,18:19:16,WSB right now analyzing GME charts.. We still ...,1,0


In [22]:
#dropping further 1 dec till 15 dec post EDA analysis
post_df= post_df[~(post_df['created_et_date'] < '2020-12-15')]
#creating coloumn for gme timeline
post_df['gme_timeline'] = np.select(
    [
        post_df['created_et_date'] < '2021-01-13',
        (post_df['created_et_date'] >= '2021-01-13') & (post_df['created_et_date'] <= '2021-01-27'),
        post_df['created_et_date'] > '2021-01-27'
    ],
    [0, 1, 2]
)
print(post_df.shape)
post_df.to_csv('out_data/posts output folder/clean_posts_with_trade_time_23_09_2024.csv', index=False)

(871818, 12)


In [23]:
#dropping non trade days and non trade time 
trade_post_df= post_df[(post_df['is_trade_day'] != 0) & (post_df['is_trade_time'] != 0)]
trade_post_df= trade_post_df.drop(columns=['is_trade_day', 'is_trade_time'])
print(trade_post_df.shape)
trade_post_df.to_csv('out_data/posts output folder/posts_onlytrade_28_09_2024.csv', index=False)

(346994, 10)


In [24]:
trade_post_df['gme_timeline'].value_counts()

gme_timeline
2    268426
1     68556
0     10012
Name: count, dtype: int64

In [52]:
trade_post_df[trade_post_df["created_et_date"] >= "2021-01-28"]["num_comments"].sum()

1904771

In [54]:
trade_post_df.loc[trade_post_df["score"].idxmax()]

created_utc                                            1611859843
id                                                         l752de
upvote_ratio                                                 0.94
num_comments                                                 4657
score                                                      231278
created_et_date                               2021-01-28 00:00:00
created_et_timestamp                    2021-01-28 13:50:43-05:00
created_et_time                                          13:50:43
fulltext                You get what you f**king deserve! UPVOTE!
gme_timeline                                                    2
Name: 349314, dtype: object